In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import base64
import io

In [25]:
app = dash.Dash()

app.layout = html.Div([
    # Title
    html.Div(
        children=[
            html.H1('Video Analysis',
                    style={'textAlign': 'center'}),
        ]),

    # Upload Video
    html.Div(
        dcc.Upload(
            id='upload-video',
            children=html.Div([
                'Drag and Drop or Select a Video',
            ]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            # Allow multiple files to be uploaded        
            multiple=True,
        ),
        style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}
    ),

    # Output Video
    html.Div(id='output-video',
             style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'})
])

@app.callback(
    Output('output-video', 'children'),
    Input('upload-video', 'contents'),
    prevent_initial_call=True
)
def update_output(list_of_contents):
    if list_of_contents is not None:
        children = [
            html.Video(src=contents, 
                       controls=True, 
                       style={'width': '50%'}) for contents in list_of_contents
        ]
        return children

if __name__ == '__main__':
    app.run_server(debug=True)